# Draft Prep

Lots prepare csv files for draft activity. Also prepare weighted csv files for other uses.

In [91]:
import os
import sys
import warnings
import re

import pandas as pd

sys.path.append(r'/home/lenhart/Repos/phi-utils')

from philosofool.data_sources.utils import read_yml
from fantasy_baseball_draft.utils import StatSynonyms, load_cbs_data, DataLoader

data_path = read_yml('local/config.yml')['paths']['local_data']


In [92]:
class AssociatePlayers:
    """Associate entities from different datasets.

    Given two dataframes that share entities but not a common single
    join key, this can construct a merge of the data based on columns
    that are assumed to find unique one.

    Example: Given dataframes of pitchers in 2019 that use different
    naming conventions ('Mike', 'Michael', etc.) we can align these by 
    assuming that two pitchers are the same if they started the same
    number of games, had the same number of walks, hits and strikeouts.
    """
    syns = StatSynonyms()

    def associate(self, df1, df2, index_cols):
        if df1.duplicated().sum():
            warnings.warn("Found duplicated in df1.")
        if df2.duplicated().sum():
            warnings.warn("Found duplicated in df2.")
        df1 = self.syns.normalize_df(df1)
        df2 = self.syns.normalize_df(df2)
        df = df1.set_index(index_cols).merge(df2.set_index(index_cols), left_index=True, right_index=True)
        return df


In [93]:
associate = AssociatePlayers()
#

index_cols = ['H', 'BB', 'RBI', 'K']
index_cols = ['IP', 'W', 'G', 'K']

In [94]:
# syns = AssociatePlayers.syns

loader = DataLoader(os.path.join(data_path, 'historical'))
fg22 = loader.load_csv('fg_hitters_2022.csv')
cbs22 = loader.load_cbs_csv('cbs_hitters_2022.csv')
fg22[fg22.Name.str.contains('France')]
#cbs22[cbs22.Player.str.contains('France')]

,Name,Team,G,AB,PA,H,1B,2B,3B,HR,...,IBB,K,HBP,SF,SH,GDP,SB,CS,BA,playerid
96,Ty France,SEA,140,551,613,151,103,27,1,20,...,3,94,21,5,0,18,0,0,0.274,17982


In [95]:
def build_id_map(cbs_path, fg_path, index_cols, duplicated=False):
    data_path = read_yml('local/config.yml')['paths']['local_data']
    hist_path = os.path.join(data_path, 'historical')
    loader = DataLoader(hist_path)
    cbs = loader.load_cbs_csv(cbs_path)
    fg = loader.load_csv(fg_path)
    
    cbs = cbs[cbs[index_cols].sum(axis=1) > 0]
    org = len(cbs)
    if duplicated:
        return cbs[cbs.duplicated(subset=index_cols, keep=False)]
    cbs = cbs.drop_duplicates(subset=index_cols, keep=False)
    print(f"Dropped {org - len(cbs)} records.")

    cbs_to_fg = associate.associate(cbs, fg, index_cols)
    cbs_to_fg = dict(zip(cbs_to_fg.Player.str.strip(), cbs_to_fg.playerid))
    return cbs_to_fg

# import json
# with open(data_path + '/hitter_ids.json', 'w') as j:
#     j.write(json.dumps(cbs_to_fg_h))

pitcher_ids = build_id_map('cbs_pitchers_2022.csv', 'fg_pitchers_2022.csv', ['IP', 'W', 'G', 'K'])
hitter_ids = build_id_map('cbs_hitters_2022.csv', 'fg_hitters_2022.csv', ['H', 'BB', 'RBI', 'K'])
#build_id_map('cbs_hitters_2022.csv', 'fg_hitters_2022.csv', ['H', 'BB', 'RBI', 'K'], True)
{k: v for k, v in hitter_ids.items() if 'Franc' in k}# [hitter_ids..str.contains('Franc')]

Dropped 34 records.
Dropped 33 records.


{'Francisco Alvarez C | NYM': 26121,
 'Franchy Cordero 1B | BAL': 14567,
 'Francisco Mejia C | TB': 16403,
 'Maikel Franco 3B | WAS': 12179,
 'Wander Franco SS | TB': 23667,
 'Francisco Lindor SS | NYM': 12916}

In [96]:
projections = DataLoader(os.path.join(data_path, 'projections'))
cbs_hitters = projections.load_cbs_csv('cbs_hitters.csv')
cbs_pitchers = projections.load_cbs_csv('cbs_pitchers.csv')
print(data_path)
fg_hitters = projections.load_csv('fg-depth-chart_hitters.csv')
fg_pitcher = projections.load_csv('fg-depth-chart_pitchers.csv')

cbs_hitters['playerid'] = cbs_hitters.Player.map(hitter_ids).fillna(-1).astype(int)
cbs_pitchers['playerid'] = cbs_pitchers.Player.map(pitcher_ids).fillna(-1).astype(int)
cbs_hitters['PA'] = cbs_hitters.AB + cbs_hitters.BB

/home/lenhart/Dropbox/baseball/fantasy_data


In [97]:
fg_hitters[fg_hitters.Name.str.contains("Tatis")]

,Name,Team,G,PA,AB,H,1B,2B,3B,HR,...,BsR,Fld,Off,Def,WAR,ADP,InterSD,InterSK,IntraSD,playerid
16,Fernando Tatis Jr.,SDP,123,532,466,128,66,24,2,36,...,2.77372,-2.36711,39.055023,-8.567107,5.08047,17.870001,NaN,NaN,NaN,19709


In [98]:
hitter_ids['Ty France 1B | SEA'] = 17982
hitter_ids["Fernando Tatis SS | SD"] = 19709

In [99]:
cbs_hitters

,Avail,Player,AB,R,H,1B,2B,3B,HR,RBI,BB,K,SB,CS,BA,OBP,SLG,Rank,playerid,PA
0,You Drink Bitches' Wine,Aaron Judge CF | NYY,555,119,170,91,30,0,49,116,89,162,14,3,0.306,0.405,0.625,1,15640,644
1,Walk-In Clostes,Jose Ramirez 3B | CLE,578,98,173,88,44,5,36,123,73,85,23,6,0.299,0.380,0.580,2,13510,651
2,Walk-In Clostes,Trea Turner SS | PHI,604,94,192,132,35,4,21,84,33,122,32,5,0.318,0.355,0.493,3,16252,637
3,Walk-In Clostes,Julio Rodriguez CF | SEA,592,96,170,100,35,5,30,85,46,167,25,8,0.287,0.349,0.515,4,23697,638
4,Walk-In Clostes,Ronald Acuna RF | ATL,545,91,151,96,32,1,22,66,62,141,31,12,0.277,0.363,0.461,5,18401,607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3847,W,Victor Roache LF | MIL,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,9999,-1,0
3848,W,Troy Johnston 1B | MIA,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,9999,-1,0
3849,W,Micker Adolfo RF | CHW,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,9999,-1,0
3850,W,Evan White 1B | SEA,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,9999,-1,0


In [100]:
cbs_pitchers['ER'] = cbs_pitchers.ERA * cbs_pitchers.IP / 9

In [101]:
def merge_fg(cbs, base_path, cols, name_filter=lambda x: x):
    cbs = cbs.copy()
    ids = cbs.playerid.to_numpy()
    loader = DataLoader(base_path)
    for path in os.listdir(base_path):
        if name_filter(path) and 'cbs' not in path:
            #print(path)
            suffixes = ('', '_' + path[:5].strip('_'))
            df = loader.load_csv(path)
            if len(df) == 0:
                #print(f'{path} data is empty...')
                continue
            #print(df.columns, df.playerid[:10])
            df = df[~df.playerid.astype(str).str.startswith('sa')]

            df.playerid = df.playerid.astype(int)
            df = df[df.playerid.isin(list(ids))]
            assert len(df) > 0, 'df empty, which is sort of like wdf?'
            cbs = cbs.merge(df[cols], on='playerid', suffixes=suffixes, how='left')
    return cbs

def merge_fg_hitters(cbs, base_path):
    cols = ['playerid', 'PA',  'AB', 'H', 'HR', 'R', 'RBI', 'SB']
    return merge_fg(cbs, base_path, cols, lambda x: 'hitter' in x)

def merge_fg_pitchers(cbs, base_path):
    cols = ['playerid', 'IP', 'ER', 'WHIP', 'K', 'S', 'W']
    return merge_fg(cbs, base_path, cols, lambda x: 'pitch' in x)

merge_fg_hitters(cbs_hitters, data_path + '/projections')
merge_fg_pitchers(cbs_pitchers, data_path + '/projections')
#cbs_pitchers

,Avail,Player,IP,G,GS,QS,CG,W,L,S,...,WHIP_zips,K_zips,S_zips,W_zips,IP_steam,ER_steam,WHIP_steam,K_steam,S_steam,W_steam
0,SweepTheLegJohnny,Shohei Ohtani DH | LAA,165,25,25,18,0,14,6,0,...,1.070510,193.0,0.0,13.0,173.0,62.0,1.10357,217.0,0.0,12.0
1,Droitwich Murdercocks,Corbin Burnes SP | MIL,178,30,30,21,0,10,7,0,...,1.049910,215.0,0.0,12.0,197.0,69.0,1.09431,240.0,0.0,13.0
2,SweepTheLegJohnny,Gerrit Cole SP | NYY,191,31,31,22,1,15,8,0,...,1.035450,224.0,0.0,14.0,197.0,71.0,1.06028,248.0,0.0,14.0
3,Strike Four,Max Scherzer SP | NYM,186,30,30,21,0,14,6,0,...,1.019110,172.0,0.0,10.0,191.0,71.0,1.04863,228.0,0.0,13.0
4,Marjorie Taylor Greene's Only Fans Page,Justin Verlander SP | NYM,178,29,29,22,0,16,5,0,...,0.963636,177.0,0.0,14.0,180.0,67.0,1.06439,202.0,0.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3174,W,Joseph Zanghi RP | NYM,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3175,W,Jacob Webb RP | LAA,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3176,W,Wei-Chung Wang RP | PIT,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3177,W,McCade Brown SP | COL,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
def add_weighted_stats(df, stats, playtime_stat):
    df = df.copy()
    pt = playtime_stat
    #print(pt)
    suffixes = set(['_'.join(col.split('_')[1:]) for col in df.columns if len(col.split('_')) > 1])
    assert (f'{pt}_fg-de' in df.columns)
    #print(df[f'{pt}_fg-de'])
    #print(df[pt].dtype)
    df[f'{pt}_adj'] = (df[f'{pt}_fg-de'] + df[pt]) / 2

    for suf in suffixes:
        for col in stats:
            df[f'{col}_{suf}'] = df[f'{col}_{suf}'] / df[f'{pt}_{suf}'] * df[f'{pt}_adj']
    for col in stats:
        adj_cols = [col] + [c for c in df.columns if re.match(f'{col}_', c)]
        df[f'{col}_adj'] = (df[adj_cols].sum(axis=1) / df[adj_cols].notna().sum(axis=1))
    return df

#add_weighted_stats(cbs_hitters, cols[2:], 'PA')
hitter_stats = ['playerid', 'PA',  'AB', 'H', 'HR', 'R', 'RBI', 'SB']
pitcher_stats = ['playerid', 'IP', 'ER', 'WHIP', 'K', 'S', 'W']
full_hitters = (cbs_hitters
    .pipe(merge_fg_hitters, data_path + '/projections')
    .pipe(add_weighted_stats, hitter_stats[2:], 'PA')
    )
full_pitchers = (cbs_pitchers
    .pipe(merge_fg_pitchers, data_path + '/projections')
    .pipe(add_weighted_stats, pitcher_stats[2:], 'IP')
)


In [103]:
from fantasy_baseball_draft.spg import FantasyValuator, spgs_from_standings_html

valuator = FantasyValuator(spgs_from_standings_html(os.path.join(data_path, 'standings/cbs_2021_standings.html')))


In [106]:
elig = DataLoader(os.path.join(data_path, 'eligibility')).load_cbs_csv('eligibility.csv')

def extract_projection(df):
    extract = ['Avail', 'Player', 'playerid'] + [col for col in df.columns if '_adj' in col]
    df = df[extract].rename(columns={k: re.sub('_adj', '', k) for k in extract})
    return df

hitter_proj = extract_projection(full_hitters)
hitter_proj = hitter_proj.merge(elig[['Player', 'Eligible']],  how='left', on='Player')
hitter_proj['fwar'] = valuator.valuate_hitters(hitter_proj, 16*12)

pitcher_proj = extract_projection(full_pitchers)
pitcher_proj['ERA'] = pitcher_proj.ER / pitcher_proj.IP * 9
pitcher_proj['fwar'] = valuator.valuate_pitchers(pitcher_proj, 16*12)

hitter_proj.sort_values('fwar', ascending=False).head(20)

6.321868798299237 -0.008795665509664576


,Avail,Player,playerid,PA,AB,H,HR,R,RBI,SB,Eligible,fwar
11,Walk-In Clostes,Fernando Tatis SS | SD,-1,NaN,520.000000,159.000000,40.000000,107.000000,103.000000,26.000000,"SS,U",13.664880
0,You Drink Bitches' Wine,Aaron Judge CF | NYY,15640,651.0,548.690881,155.639048,44.731453,108.897703,111.373825,10.917948,"CF,RF,U",11.300801
2,Walk-In Clostes,Trea Turner SS | PHI,16252,654.5,598.841950,177.578441,21.633607,95.556678,79.881837,28.301841,"SS,U",10.259670
4,Walk-In Clostes,Ronald Acuna RF | ATL,18401,622.0,535.073892,145.963822,28.075452,100.517090,77.151192,33.007567,"RF,U",10.119754
1,Walk-In Clostes,Jose Ramirez 3B | CLE,13510,651.0,571.360440,156.525145,30.826891,93.879379,105.123594,22.397414,"3B,U",10.073746
3,Walk-In Clostes,Julio Rodriguez CF | SEA,23697,641.0,579.948194,160.027699,29.354871,94.149862,87.990698,25.923615,"CF,U",9.697274
10,Freddie & the Mets,Freddie Freeman 1B | LAD,5361,664.0,573.815127,173.473741,25.634954,102.563525,95.237498,9.736378,"1B,U",9.301643
6,Freddie & the Mets,Kyle Tucker RF | HOU,18345,620.5,547.865760,150.616512,30.602224,86.213195,99.921124,21.325625,"RF,U",9.262518
13,Omak Wrong Players,Vladimir Guerrero 1B | TOR,19611,639.5,566.849436,164.887096,35.296484,94.866169,103.136167,6.314437,"1B,U",9.056083
9,Omak Wrong Players,Juan Soto RF | SD,20123,679.5,533.764584,149.394083,32.475459,111.996352,90.957444,9.405037,"RF,U",8.735556


In [107]:
hitter_proj.to_csv('local/hitter_proj.csv', index_label='index')
pitcher_proj.to_csv('local/pitcher_proj.csv', index_label='index')